In [2]:
import numpy as np
import nltk
nltk.download('punkt')
import pandas as pd
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import string
import re
from math import log
nltk.download('stopwords')
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import math

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spyro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#Preprocessing function

def preprocessing(df):
    
    c=[]
    for i in range(len(df)):
        text=str(df[i])
        #print(text, i)
        
        #punctuation removal , lowercase
        text=re.sub('[^a-zA-Z0-9]',' ',text)
        text=re.sub('\s([?.!"](?:\s|$))', r'\1',text)
        text=str(text).lower()
        text=word_tokenize(text)
        
        
        # lemmatisation
        lemmatiser = WordNetLemmatizer()
        tokens = [lemmatiser.lemmatize(t) for t in text]
        tokens = [t for t in tokens if t] # ensure no empty space
        
        # stopword removal
        stop = set(stopwords.words('english'))
        tokens = [t for t in tokens if t not in stop]
        
        
        text=' '.join(tokens)
        c.append(text)
        #print(c)
        
    return c

In [4]:
def calculate_length(text):
    try:
        return len(text.split(' '))
    except AttributeError:
        return 0



In [6]:
#Not in use

def count_terms(df):
   
    c=[]
    for i in range(len(df)):
        counts = dict()
        words=str(df[i])
        tokens = nltk.word_tokenize(words)
        for word in tokens:
            if word in counts:
                counts[word] += 1
            else:
                counts[word] = 1
        c.append(counts)
        #print(counts)

    return c

In [23]:
#frequency of word in index, i.e. number of documents that contain word - NOT IN USE
def get_index_freq(word,df):
    index_freq=0
    for i in range(len(df)):
        if word in df['tf'][i]:
            index_freq+=1
    return index_freq

In [78]:
 def getfrequencies(corpus):
    
    tf = []
    docf = {}
    idf = {}
    doc_len = []
    corpus_size = 0
    
    for line in corpus:
        corpus_size += 1
        doc_len.append(len(line))

        # compute tf (term frequency) per document
        freq = {}
        tokens = nltk.word_tokenize(line)
        for word in tokens:
           # print(word)
            word_count = freq.get(word, 0) + 1
            freq[word] = word_count

        tf.append(freq)

        # compute df (document frequency) per term
        for word, _ in freq.items():
            docf_count = docf.get(word, 0) + 1
            docf[word] = docf_count

    for word, freq in docf.items():
        idf[word] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

    avg_doc_len = sum(doc_len) / corpus_size
    
    return doc_len,corpus_size,tf,docf,idf,avg_doc_len

In [72]:
def query_preproces(query):
    
    stop_words = set(stopwords.words('english')) 
  
    tokens = nltk.word_tokenize(query)
  
    pre = [w for w in tokens if not w in stop_words] 
  
    pre = [] 
  
    for w in tokens: 
        if w not in stop_words: 
            pre.append(w.lower()) 

    return pre

In [146]:
def rank(query, corpus,corpus_size, doc_len, tf, idf, avg_doc_len, k1, b ):
    s=[]
    for line in range(corpus_size):
        #print(line)
        doclen = doc_len[line]
        frequencies = tf[line]
        score = 0.0
        for term in query:
            if term in frequencies:
        
                freq = frequencies[term]
                numerator = idf[term] * freq * (k1 + 1)
                denominator = freq + k1 * (1 - b + b * doclen / avg_doc_len)
                score += (numerator / denominator)
        s.append(score)

    return s

In [7]:
#Create the pandas dataframe
df = pd.read_csv(r"articles.csv",encoding='utf-8')
df.dropna()
df=df[:1000]

#get unique id's
for i in range(len(df)):
    df['id'][i] = i+1

#Preprocess the data and get the length
df['processed_title'] = preprocessing(df['title'])
df['processed_content'] = preprocessing(df['content'])
df['length'] = df['processed_content'].apply(lambda x: calculate_length(x))

#count the terms
#df['count_terms']= count_terms(df['processed_content'])




C:\Users\spyro\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [105]:
corpus=[]

for i in range(len(df)):
    corpus.append(df['processed_content'][i])
    
doc_len,corpus_size,tf,doc_freq,idf,avg_doc_len = getfrequencies(corpus)

df['tf']= tf

In [139]:
query = 'Trump Presentent of America?'
query = query_preproces(query)
query


['trump', 'presentent', 'america', '?']

In [147]:
k1=1.5
b=0.75

scor = rank(query, corpus,corpus_size, doc_len, tf, idf, avg_doc_len, k1, b )


In [148]:
df['scores']= scor
df.sort_values(by=['scores'],ascending=False)
#df[['scores','title']]


,Unnamed: 0,id,title,publication,author,date,year,month,url,content,processed_title,processed_content,length,tf,scores
857,104316,858,Trump’s tax plan: massive cuts for the 1% will...,Guardian,Rupert Neate,2016-11-23,2016.0,11.0,https://www.theguardian.com/us-news/2016/nov/2...,President Donald Trump is set to give America’...,trump tax plan massive cut 1 usher era dynasti...,president donald trump set give america riches...,569,"{'president': 2, 'donald': 1, 'trump': 19, 'se...",5.942126
954,104413,955,"Election diary: Trump as Stalin, the future of...",Guardian,David Smith,2016-10-22,2016.0,10.0,https://www.theguardian.com/us-news/2016/oct/2...,Mark Wednesday in your diary. The Trump family...,election diary trump stalin future tv wikileak...,mark wednesday diary trump family host officia...,519,"{'mark': 1, 'wednesday': 4, 'diary': 1, 'trump...",5.927724
963,104422,964,The Guardian view on America’s choice: Don’t v...,Guardian,Editorial,2016-11-07,2016.0,11.0,https://www.theguardian.com/commentisfree/2016...,It is make your mind up time for America. A ho...,guardian view america choice vote trump elect ...,make mind time america hopeful incrementally b...,421,"{'make': 3, 'mind': 1, 'time': 5, 'america': 6...",5.912647
873,104332,874,Mark Cuban insults Trump before backing ’true ...,Guardian,Lauren Gambino,2016-07-30,2016.0,7.0,https://www.theguardian.com/us-news/2016/jul/3...,The battle of the brash billionaires is on. En...,mark cuban insult trump backing true leader hi...,battle brash billionaire entrepreneur mark cub...,347,"{'battle': 1, 'brash': 1, 'billionaire': 1, 'e...",5.742731
398,103857,399,Trump deletes tweet with image of the star of ...,Guardian,Jamiles Lartey,2016-07-02,2016.0,7.0,https://www.theguardian.com/us-news/2016/jul/0...,Donald Trump deleted an image of Hillary Clint...,trump deletes tweet image star david hillary c...,donald trump deleted image hillary clinton sta...,271,"{'donald': 1, 'trump': 9, 'deleted': 1, 'image...",5.692855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,103874,416,"Leicester’s woes continue as Hazard, Costa and...",Guardian,Dominic Fifield,2016-10-15,2016.0,10.0,https://www.theguardian.com/football/2016/oct/...,There was a moment just after the here when ...,leicester woe continue hazard costa moses fire...,wa moment frustration finally overcame claudio...,439,"{'wa': 13, 'moment': 1, 'frustration': 1, 'fin...",0.000000
416,103875,417,French woman accused of murdering daughter on ...,Guardian,Kim Willsher,2016-06-20,2016.0,6.0,https://www.theguardian.com/world/2016/jun/20/...,A French woman who left her baby daughter to d...,french woman accused murdering daughter beach ...,french woman left baby daughter drown beach bl...,351,"{'french': 3, 'woman': 1, 'left': 2, 'baby': 1...",0.000000
417,103876,418,Standing Rock: injured protester’s father says...,Guardian,Julia Carrie Wong,2016-11-23,2016.0,11.0,https://www.theguardian.com/us-news/2016/nov/2...,North Dakota law enforcement have blamed prote...,standing rock injured protester father say pol...,north dakota law enforcement blamed protester ...,523,"{'north': 6, 'dakota': 8, 'law': 11, 'enforcem...",0.000000
421,103880,422,Fox News is a cesspool of sexism. Firing Roger...,Guardian,Jessica Valenti,2016-07-19,2016.0,7.0,https://www.theguardian.com/commentisfree/2016...,"This week, all signs point to Roger Ailes bein...",fox news cesspool sexism firing roger ailes fix,week sign point roger ailes fired position hea...,277,"{'week': 1, 'sign': 1, 'point': 1, 'roger': 1,...",0.000000
